In [1]:
args_pre = {
    'det_config': 'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py',
    'det_checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth',
    'pose_detector_config': 'configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/hrnet_w48_coco_wholebody_256x192.py',
    'pose_detector_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_256x192-643e18cb_20200922.pth',
    'pose_lifter_config': 'configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/combined/videopose3d_combined_243frames_fullconv_supervised.py',
    'pose_lifter_checkpoint': '/home/ubuntu/epoch_160.pth',
    # Flags/Optional
#     'video_path': 'demo/resources/jeldwen-1.mp4',
    'rebase_keypoint_height': True,
    'norm_pose_2d': None,
    'num_instances': -1,
    'show': False,
    'out_video_root': 'vis_results',
    'device': 'cuda:0',
    'det_cat_id': 1,
    'bbox_thr': 0.9,
    'kpt_thr': 0.3,
    'use_oks_tracking': None,
    'tracking_thr': 0.3,
    'euro': None,
    'radius': 8,
    'thickness': 2,
}
from types import SimpleNamespace
args = SimpleNamespace(**args_pre)

In [2]:
from demo import zach_v_body3d_two_stage_video as m_2d_3d_model 

In [ ]:
# Test one video

args.video_name = 'LENNOX_occlusion_arm1'
args.file_path = 'demo/resources/test_vids/LENNOX_occlusion_arm1.mp4'
args.detections_2d = ''
m_2d_3d_model.process_video(args)

In [3]:
import glob
import re

video_directory = 'test_vids'

def get_video_list():
    '''Get list of videos to process'''
    file_paths = glob.glob("demo/resources/{}/*".format(video_directory))
    video_names = []
    vid_list = ['LENNOX_occlusion_arm1',
         'BERKLAB_bad_angle1_1',
         'SENTRY_hook_bagger_1',
         'SENTRY_conveyor1',
            'BAR-S_water_ballon', 
          'MEMIC_profile_view_lab_coat_legs_missing_1', 
          'AFG_subject_switching_close_nurses',
         'BARS_hotdog_packing',
        'MERCER_box_slamming1']
    for path in file_paths:
        flag = False
        for p in vid_list:
            if (p in path):
                flag = True
        if (not flag): continue
        video_names.append(
            SimpleNamespace(
                **{
                    'video_name': re.search('/.*/.*/(.*)(?:[.])', path).group(1),
                    'file_path': 'demo/resources/{}/'.format(video_directory) + re.search('/.*/.*/(.*)', path).group(1)
                }
            )
        )
    
    return video_names


In [4]:
def process_videos():
    '''Iterate through videos and process'''
    video_list = get_video_list()
    print(video_list)
    for video in video_list:
        args.video_name = video.video_name
        args.file_path = video.file_path
        args.detections_2d = ''
        m_2d_3d_model.process_video(args)
        
process_videos()

[namespace(file_path='demo/resources/test_vids/SENTRY_hook_bagger_1.mov', video_name='SENTRY_hook_bagger_1'), namespace(file_path='demo/resources/test_vids/SENTRY_conveyor1.mov', video_name='SENTRY_conveyor1'), namespace(file_path='demo/resources/test_vids/AFG_subject_switching_close_nurses_subjects1.MOV', video_name='AFG_subject_switching_close_nurses_subjects1'), namespace(file_path='demo/resources/test_vids/MEMIC_profile_view_lab_coat_legs_missing_1.mov', video_name='MEMIC_profile_view_lab_coat_legs_missing_1'), namespace(file_path='demo/resources/test_vids/BAR-S_water_ballon.mov', video_name='BAR-S_water_ballon'), namespace(file_path='demo/resources/test_vids/AFG_subject_switching_close_nurses_subjects1_partial.MOV', video_name='AFG_subject_switching_close_nurses_subjects1_partial'), namespace(file_path='demo/resources/test_vids/LENNOX_occlusion_arm1.mp4', video_name='LENNOX_occlusion_arm1'), namespace(file_path='demo/resources/test_vids/MERCER_box_slamming1.mp4', video_name='MERCE

load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_256x192-643e18cb_20200922.pth
Initialized Model
pose_det_dataset-> TopDownCocoWholeBodyDataset
Video len: 1176
Video len: 1176
Time: 0.4801027774810791
Time: 0.1875615119934082
Time: 0.18153810501098633
Time: 0.18116521835327148
Time: 0.18157649040222168
Time: 0.1802365779876709
Time: 0.18173837661743164
Time: 0.180833101272583
Time: 0.18028688430786133
Time: 0.17959952354431152
Time: 0.18127226829528809
Time: 0.17893695831298828
Time: 0.18241405487060547
Time: 0.183915376663208
Time: 0.1822528839111328
Time: 0.18224549293518066
Time: 0.18221735954284668
Time: 0.18143796920776367
Time: 0.18236708641052246
Time: 0.1824178695678711
Time: 0.18384790420532227
Time: 0.18164396286010742
Time: 0.18413519859313965
Time: 0.18167877197265625
Time: 0.17983102798461914
Time: 0.18103885650634766
Time: 0.18185997009277344
Time: 0.18099355697631836
Time: 0.18161654472351074
Time: 0.1816031932

Time: 0.18265962600708008
Time: 0.18193364143371582
Time: 0.18108415603637695
Time: 0.18253612518310547
Time: 0.18266010284423828
Time: 0.18442559242248535
Time: 0.18417668342590332
Time: 0.1814429759979248
Time: 0.18415236473083496
Time: 0.1835799217224121
Time: 0.18464922904968262
Time: 0.18736982345581055
Time: 0.18463397026062012
Time: 0.18141388893127441
Time: 0.18218421936035156
Time: 0.18364858627319336
Time: 0.1827986240386963
Time: 0.18005967140197754
Time: 0.18237948417663574
Time: 0.18189501762390137
Time: 0.18128061294555664
Time: 0.1831221580505371
Time: 0.1804358959197998
Time: 0.1811833381652832
Time: 0.18323755264282227
Time: 0.18123197555541992
Time: 0.18126177787780762
Time: 0.18251538276672363
Time: 0.18074393272399902
Time: 0.18190240859985352
Time: 0.18270611763000488
Time: 0.18109512329101562
Time: 0.18175506591796875
Time: 0.18332600593566895
Time: 0.18209266662597656
Time: 0.18213725090026855
Time: 0.18208646774291992
Time: 0.18137049674987793
Time: 0.1809394359

Time: 0.18042731285095215
Time: 0.1828758716583252
Time: 0.18107390403747559
Time: 0.1814265251159668
Time: 0.18388104438781738
Time: 0.18194174766540527
Time: 0.18157577514648438
Time: 0.1814744472503662
Time: 0.18241262435913086
Time: 0.18077826499938965
Time: 0.18315744400024414
Time: 0.18097615242004395
Time: 0.1813504695892334
Time: 0.18204665184020996
Time: 0.18146800994873047
Time: 0.18125295639038086
Time: 0.18384528160095215
Time: 0.1808757781982422
Time: 0.18329286575317383
Time: 0.18412089347839355
Time: 0.1833498477935791
Time: 0.18096089363098145
Time: 0.18267822265625
Time: 0.17507624626159668
Time: 0.17550969123840332
Time: 0.177001953125
Time: 0.1739823818206787
Time: 0.17550063133239746
Time: 0.17668843269348145
Time: 0.17598962783813477
Time: 0.17511224746704102
Time: 0.17859435081481934
Time: 0.17433977127075195
Time: 0.1754927635192871
Time: 0.17731022834777832
Time: 0.17831754684448242
Time: 0.17662739753723145
Time: 0.17604732513427734
Time: 0.17703914642333984
Ti

Time: 0.17622876167297363
Time: 0.17356085777282715
Time: 0.1759629249572754
Time: 0.17623662948608398
Time: 0.17384600639343262
Time: 0.1743624210357666
Time: 0.17606663703918457
Time: 0.1750326156616211
Time: 0.17490053176879883
Time: 0.17641615867614746
Time: 0.17534375190734863
Time: 0.17521905899047852
Time: 0.17638254165649414
Time: 0.17609810829162598
Time: 0.1738438606262207
Time: 0.17593646049499512
Time: 0.17366981506347656
Time: 0.1744823455810547
Time: 0.17572283744812012
Time: 0.17492103576660156
Time: 0.17513608932495117
Time: 0.1751401424407959
Time: 0.17343902587890625
Time: 0.17529988288879395
Time: 0.1773543357849121
Time: 0.1751861572265625
Time: 0.1766376495361328
Time: 0.17759251594543457
Time: 0.17521119117736816
Time: 0.17433834075927734
Time: 0.17615079879760742
Time: 0.1751880645751953
Time: 0.17672181129455566
Time: 0.17812776565551758
Time: 0.1743016242980957
Time: 0.17511439323425293
Time: 0.17732906341552734
Time: 0.17477107048034668
Time: 0.174056053161621

...

[>>>>>>>>>>>>>>>>>>>>>>     ] 2139/2608, 32.2 task/s, elapsed: 67s, ETA:    15sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2140/2608, 32.2 task/s, elapsed: 67s, ETA:    15sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2141/2608, 32.2 task/s, elapsed: 67s, ETA:    15sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2142/2608, 32.2 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2143/2608, 32.2 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2144/2608, 32.2 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2145/2608, 32.1 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2146/2608, 32.1 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2147/2608, 32.1 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2148/2608, 32.1 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2149/2608, 32.1 task/s, elapsed: 67s, ETA:    14sLen 243
[>>>>>>>>>>>>>>>>>>>>>>     ] 2150/2608, 32

[>>>>>>>>>>>>>>>>>>>>>>>    ] 2233/2608, 32.3 task/s, elapsed: 69s, ETA:    12sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2234/2608, 32.3 task/s, elapsed: 69s, ETA:    12sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2235/2608, 32.3 task/s, elapsed: 69s, ETA:    12sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2236/2608, 32.3 task/s, elapsed: 69s, ETA:    12sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2237/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2238/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2239/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2240/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2241/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2242/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2243/2608, 32.4 task/s, elapsed: 69s, ETA:    11sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>    ] 2244/2608, 32

[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2327/2608, 33.0 task/s, elapsed: 70s, ETA:     9sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2328/2608, 33.0 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2329/2608, 33.0 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2330/2608, 33.0 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2331/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2332/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2333/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2334/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2335/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2336/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2337/2608, 33.1 task/s, elapsed: 71s, ETA:     8sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>   ] 2338/2608, 33

[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2421/2608, 33.5 task/s, elapsed: 72s, ETA:     6sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2422/2608, 33.5 task/s, elapsed: 72s, ETA:     6sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2423/2608, 33.5 task/s, elapsed: 72s, ETA:     6sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2424/2608, 33.5 task/s, elapsed: 72s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2425/2608, 33.5 task/s, elapsed: 72s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2426/2608, 33.5 task/s, elapsed: 72s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2427/2608, 33.5 task/s, elapsed: 72s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2428/2608, 33.5 task/s, elapsed: 72s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2429/2608, 33.5 task/s, elapsed: 73s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2430/2608, 33.5 task/s, elapsed: 73s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2431/2608, 33.5 task/s, elapsed: 73s, ETA:     5sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>  ] 2432/2608, 33

[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2515/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2516/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2517/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2518/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2519/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2520/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2521/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2522/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2523/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2524/2608, 33.5 task/s, elapsed: 75s, ETA:     3sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2525/2608, 33.5 task/s, elapsed: 75s, ETA:     2sLen 243
[>>>>>>>>>>>>>>>>>>>>>>>>>> ] 2526/2608, 33

In [ ]:
# Test loading pickeled keypoints
import pickle
with open ('work_dirs/tumeke_testing/pickle_files/BAR-S_water_ballon.p', 'rb') as fp:
    raw_data = pickle.load(fp)

'''Load picked data into numpy.
    
Bounding box values are: 'xyxy' = (left, top, right, bottom)

Key Point values are: (ndarray[Kx3]): x, y, score

'''    

raw_data

In [ ]:
#Just focus on videos with one subject for now

raw_data[0]

In [ ]:
# View histogram of subjects per frame
# subjects = [len(i) for i in raw_data]
# plt.hist(subjects, bins=200)

In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})


# def index_of(val, in_list):
#     try:
#         return in_list.index(val)
#     except ValueError:
#         return -1 

def pull_from_dict(key, dict):
    try:
        return dict[key]
    except (KeyError, TypeError):
        if key == 'bbox':
            return np.full(5,np.NaN)
        else:
            return np.NaN

def extract_subject(subject_num, subjects):
    try:
        return subjects[subject_num-1]
    except IndexError:
        return None 

def raw_data_to_dataframe(raw_data):
    """ Load into dataframe"""

    # Get first identified subject in every frame
    first_subj = np.array([extract_subject(1, i) for i in raw_data])

    # Extract relevant fields to be set as columns
    bbox = np.array([pull_from_dict('bbox', i) for i in first_subj])
    area = np.array([pull_from_dict('area', i) for i in first_subj])
    track_id = np.array([pull_from_dict('track_id', i) for i in first_subj])

    # Structure Keypoints
    keypoints_array = []
    for frame in first_subj:
        
        #TODO(znoland):sanity check this and make sure no issues!
        if not frame:
            # If no data for subject
            frame_keypoints = {}
            for index in range(17):
                frame_keypoints['j{}_x'.format(index)] = np.NaN
                frame_keypoints['j{}_y'.format(index)] = np.NaN
                frame_keypoints['j{}_score'.format(index)] = np.NaN
            keypoints_array.append(frame_keypoints)
            
        else:
            # If data for subject
            frame_keypoints = {}
            for index, keypoint in enumerate(frame['keypoints']):
                frame_keypoints['j{}_x'.format(index)] = keypoint[0]
                frame_keypoints['j{}_y'.format(index)] = keypoint[1]
                frame_keypoints['j{}_score'.format(index)] = keypoint[2]
            keypoints_array.append(frame_keypoints)

    # Create seperate dataframes (otherwise can't combine as >1-dimensional fields in pandas or numpy)
    bbox_s = pd.DataFrame(bbox, columns=['bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom', 'bbox_score'])
    area_s = pd.DataFrame(area, columns=['area'])
    track_id_s = pd.DataFrame(track_id, columns=['track_id'])
    keypoints_s = pd.DataFrame(keypoints_array)


    # Create combined DataFrame
    df = pd.concat([bbox_s,area_s,track_id_s,keypoints_s], axis=1)

    return df

    #TODO(znoland): Set index as a frame number column (e.g. video1_frame_num)
    #TODO(znoland): Add column with the name of the video?

    
df = raw_data_to_dataframe(raw_data)
df

In [ ]:
# Create column names
score_column_names = ['bbox_score']
for i in range(17):
    score_column_names.append('j{}_score'.format(i))

pd.DataFrame(df[score_column_names].median()).T

### Visualize Scores - Hist

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.show()
sns.set(rc={'figure.figsize':(21,15)})

import matplotlib.pyplot as plt
n_rows=3
n_cols=6
# Create the subplots
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols)

# Set x axis limit (keep all charts on same axis)
for row in axes:
    for chart in row:
        chart.set_xlim(0,1) 

for i, column in enumerate(df[score_column_names].columns):
    sns.histplot(df[column],ax=axes[i//n_cols,i%n_cols])

In [ ]:
def euclidean_dist(p1, p2):
    # Euclidean distance
    return np.linalg.norm(p1 - p2)


In [ ]:
# # initializing points in
# # numpy arrays
# point1 = np.array((1, 2))
# point2 = np.array((2, 4))
 
# # calculating Euclidean distance
# # using linalg.norm()
# dist = np.linalg.norm(point1 - point2)
# dist

In [ ]:
def f2d_skeleton_smoothness(df):
    ''' Get Euclidean distance '''
    
    # Build dict to accumulate calcs
    dict_2d_smth = {}
    for j in range(17):
            joint_field = 'j{}_euc_dist'.format(j)
            dict_2d_smth[joint_field] = []

    # Iterate through rows (Standard shift(-1) can't be used with this calc, so must iterate)
    for i in range(len(df)):
        for j in range(17):
            joint_field_x = 'j{}_x'.format(j)
            joint_field_y = 'j{}_y'.format(j)
            joint_field_score = 'j{}_score'.format(j)
            
            #TODO(znoland): return NaN when score below certain amount?
            
            try:
                dict_2d_smth['j{}_euc_dist'.format(j)].append(
                    euclidean_dist(
                        np.array([df.loc[i, joint_field_x], df.loc[i, joint_field_y]]), 
                        np.array([df.loc[i+1, joint_field_x], df.loc[i+1, joint_field_y]])
                    )
                )
            except:
                print('skiped row {}'.format(i))

    return pd.DataFrame(dict_2d_smth)

df_2d_smth = f2d_skeleton_smoothness(df)
df_2d_smth


### Visualize Sample Euclidean Distance - Hist

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.show()
sns.set(rc={'figure.figsize':(21,15)})

# df_2d_smth.j0_euc_dist.hist()
# sns.histplot(df_2d_smth).show()
    
import matplotlib.pyplot as plt
n_rows=3
n_cols=6
# Create the subplots
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols)

# Set x axis limit (keep all charts on same axis)
for row in axes:
    for chart in row:
        chart.set_xlim(0,df_2d_smth.quantile(0.90).max()) # df_2d_smth.max().max()

for i, column in enumerate(df_2d_smth.columns):
    sns.histplot(df_2d_smth[column],ax=axes[i//n_cols,i%n_cols])


## Load example ground truth video

In [ ]:
import json

with open('work_dirs/tumeke_testing/ground_truth_labels/BAR-S_water_ballon.json') as f:
  labels = json.load(f)

In [ ]:
len(labels)

In [ ]:
labels[0]